## DB Computers (Ex's 1-13)

### Description of Computer firm DB

The database scheme consists of four tables:
- `Product` (maker, model, type)
- `PC` (code, model, speed, ram, hd, cd, price)
- `Laptop` (code, model, speed, ram, hd, screen, price)
- `Printer` (code, model, color, type, price)

The `Product` table contains data on the maker, model number, and type of product ('PC', 'Laptop', or 'Printer'). It is assumed that model numbers in the Product table are unique for all makers and product types.
 
Each personal computer in the `PC table` is unambiguously identified by a unique code, and is additionally characterized by its model (foreign key referring to the Product table), processor speed (in MHz) – speed field, RAM capacity (in Mb) - ram, hard disk drive capacity (in Gb) – hd, CD-ROM speed (e.g, '4x') - cd, and its price.

The `Laptop` table is similar to the PC table, except that instead of the CD-ROM speed, it contains the screen size (in inches) – screen. 

For each printer model in the `Printer table`, its output type (‘y’ for color and ‘n’ for monochrome) – color field, printing technology ('Laser', 'Jet', or 'Matrix') – type, and price are specified.

![Сomputers](computers.gif)

### 1. Find the model number, speed and hard drive capacity for all the PCs with prices below $500.
Result set: model, speed, hd.

In [ ]:
SELECT model, speed, hd
FROM PC
WHERE price < 500

### 2. List all printer makers. Result set: maker.

In [ ]:
SELECT DISTINCT maker as Maker
FROM Product
WHERE type = 'Printer'

### 3. Find the model number, RAM and screen size of the laptops with prices over $1000

In [ ]:
SELECT model, ram, screen
FROM laptop
WHERE price > 1000

### 4. Find all records from the Printer table containing data about color printers.

In [ ]:
SELECT *
FROM Printer
WHERE color = 'y';

### 5. Find the model number, speed and hard drive capacity of PCs cheaper than $600 having a 12x or a 24x CD drive.

In [ ]:
SELECT model, speed, hd
FROM pc
WHERE price < 600 AND (cd = '12x' OR cd = '24x');

### 6. For each maker producing laptops with a hard drive capacity of 10 Gb or higher, find the speed of such laptops. Result set: maker, speed.

In [ ]:
SELECT DISTINCT p1.maker,l1.speed
FROM product AS p1
JOIN laptop AS l1
ON p1.model = l1.model
WHERE l1.hd >= 10;

### 7. Get the models and prices for all commercially available products (of any type) produced by maker B.

In [ ]:
SELECT all_products.model, all_products.price
FROM (SELECT model, price FROM pc
      UNION
      SELECT model, price FROM laptop
      UNION
      SELECT model, price FROM printer) AS all_products
JOIN product
ON all_products.model = product.model
WHERE product.maker = 'B';

### 8. Find the makers producing PCs but not laptops.

In [ ]:
SELECT DISTINCT maker
FROM product
WHERE type IN ('pc', 'laptop')
EXCEPT
SELECT maker
FROM product
WHERE type = 'laptop';

### 9. Find the makers of PCs with a processor speed of 450 MHz or more. Result set: maker.

In [ ]:
SELECT DISTINCT p1.maker
FROM product AS p1
JOIN pc AS pc1
ON p1.model = pc1.model
WHERE pc1.speed >= 450;

### 10. Find the printer models having the highest price. Result set: model, price.

In [ ]:
SELECT DISTINCT model, price
FROM printer
WHERE price = (SELECT MAX(price) FROM printer);

### 11. Find out the average speed of PCs.

In [ ]:
SELECT AVG(speed)
FROM PC

### 12. Find out the average speed of the laptops priced over $1000.

In [ ]:
SELECT AVG(speed)
FROM laptop
WHERE price > 1000

### 13. Find out the average speed of the PCs produced by maker A.

In [ ]:
SELECT AVG(speed)
FROM pc
JOIN product
ON product.model = pc.model
WHERE product.maker = 'A';

### 15. Get hard drive capacities that are identical for two or more PCs.
Result set: hd.

In [ ]:
SELECT hd
FROM pc
GROUP BY hd
HAVING COUNT(hd) > 1;

### 16. Get pairs of PC models with identical speeds and the same RAM capacity. 
Each resulting pair should be displayed only once, i.e. (i, j) but not (j, i).
Result set: model with the bigger number, model with the smaller number, speed, and RAM.

In [ ]:
SELECT 
    t1.model AS model_bigger,
    t2.model AS model_smaller,
    t1.speed,
    t1.ram
FROM (
    SELECT DISTINCT model, speed, ram FROM pc
) AS t1
INNER JOIN (
    SELECT DISTINCT model, speed, ram FROM pc
) AS t2
ON t1.speed = t2.speed AND t1.ram = t2.ram
WHERE t1.model > t2.model
ORDER BY t1.model, t2.model

### 17. Get the laptop models that have a speed smaller than the speed of any PC.
Result set: type, model, speed.

In [ ]:
SELECT type, laptop.model, laptop.speed
FROM laptop
JOIN product
ON laptop.model = product.model
WHERE speed < (
    SELECT MIN(speed) FROM pc)

In [ ]:
SELECT DISTINCT 'Laptop', model, speed
FROM Laptop
WHERE speed < (SELECT MIN(speed) FROM PC)

### 18. Find the makers of the cheapest color printers.
Result set: maker, price.

In [ ]:
SELECT DISTINCT product.maker, printer.price
FROM printer 
JOIN product
ON printer.model = product.model 
WHERE printer.color = 'y' -- Check color printers
    AND printer.price = (
        SELECT MIN(price)
        FROM printer
        WHERE printer.color = 'y'
)

In [ ]:
SELECT DISTINCT product.maker, filtered_printers.price
FROM (
    SELECT model, price
    FROM printer
    WHERE color = 'y'
      AND price = (
          SELECT MIN(price)
          FROM printer
          WHERE color = 'y'
      )
) AS filtered_printers
JOIN product
ON filtered_printers.model = product.model;

## DB Ships (Ex's 14 -)

![Ships](ships.gif)

The database of naval ships that took part in World War II is under consideration. 

The database consists of the following relations:
- `Classes`(class, type, country, numGuns, bore, displacement)
- `Ships` (name, class, launched)
- `Battles` (name, date)
- `Outcomes`(ship, battle, result)

Ships in classes all have the same general design. A class is normally assigned either the name of the first ship built according to the corresponding design, or a name that is different from any ship name in the database. The ship whose name is assigned to a class is called a lead ship.

The `Classes` relation includes the name of the class, type (can be either bb for a battle ship, or bc for a battle cruiser), country the ship was built in, the number of main guns, gun caliber (bore diameter in inches), and displacement (weight in tons). 

The `Ships` relation holds information about the ship name, the name of its corresponding class, and the year the ship was launched. 

The `Battles` relation contains names and dates of battles the ships participated in 

The `Outcomes` relation - the battle result for a given ship (may be sunk, damaged, or OK, the last value meaning the ship survived the battle unharmed).

Notes: 
1. The Outcomes relation may contain ships not present in the Ships relation. 
2. A ship sunk can’t participate in later battles. 
3. For historical reasons, lead ships are referred to as head ships in many exercises.
4. A ship found in the Outcomes table but not in the Ships table is still considered in the database. This is true even if it is sunk.

### 14. For the ships in the Ships table that have at least 10 guns, get the class, name, and country. 

In [ ]:
SELECT sh1.class, sh1.name, cl1.country
FROM ships AS sh1
JOIN classes AS cl1
ON sh1.class = cl1.class
WHERE numGuns >= 10